This is a minimal example of using `torch.utils.cpp_extension.load_inline` to use a custom CUDA kernel in Pytorch

In [1]:
import torch, os, math
import numpy as np
from torch.utils.cpp_extension import load_inline
from pathlib import Path

In [2]:
np.set_printoptions(precision=2, linewidth=140)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

## CUDA Setup

In [3]:
torch.cuda.is_available()

True

In [4]:
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [5]:
%load_ext wurlitzer

In [6]:
def load_cuda_inline(cuda_src, cpp_src, funcs, opt=False, verbose=False, build_directory=None, name=None):
    if name is None:
        name = funcs[0]
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs,
                       extra_cuda_cflags=["-O2"] if opt else [],
                       verbose=verbose, name=name, build_directory=build_directory)

## Matmul CUDA kernel

In [7]:
cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a + b - 1) / b;}
'''

In [8]:
cuda_src = cuda_begin + r'''
__global__ void matmul_k(float* m, float* n, float* out, int h, int w, int k) {
    int r = blockIdx.y*blockDim.y + threadIdx.y;
    int c = blockIdx.x*blockDim.x + threadIdx.x;

    if (r>=h || c>=w) return;
    float o = 0;
    for (int i = 0; i<k; ++i) o += m[r*k+i] * n[i*w+c];
    out[r*w+c] = o;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k==n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(16,16);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));
    matmul_k<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
'''

In [9]:
cpp_src = "torch::Tensor matmul(torch::Tensor m, torch::Tensor n);"

In [10]:
torch.manual_seed(1)
m1 = torch.randn(1000,500).contiguous().cuda()
m2 = torch.randn(500,1000).contiguous().cuda()

In [11]:
module = load_cuda_inline(cuda_src, cpp_src, ['matmul'])

/opt/conda/envs/flash-attention/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [12]:
# Check correctness
torch.isclose(torch.matmul(m1, m2), module.matmul(m1, m2), atol=1e-4).all().cpu()

tensor(True, device='cuda:0')

In [14]:
%timeit -n 10 _= module.matmul(m1, m2).cpu()

4.03 ms ± 212 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit -n 10 _= torch.matmul(m1, m2).cpu()

2.57 ms ± 33.3 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
